# Chapter 2. Multi-armed Bandits

In [17]:
import numpy as np
import os
from tqdm import tqdm_notebook

np.random.seed(77)

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams["figure.figsize"] = (11,4)

plt.rcParams['axes.unicode_minus'] = False

PROJECT_ROOT_DIR = "."
CHAPTER_ID = '10_armed_testbed'

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

## 2.3 The 10-Armed Testbed

In [ ]:
def sample_average(t, action, actions, epsilon = 0):
    
    sum_rewards = 0
    sum_predicate = 0
    
    if t <= 1:
        return 0
   
    else:
        for i in range(t-1):
            predicate = 0
            
            reward = get_reward(action)
            
            if actions[i] == action:
                predicate = 1
            sum_rewards += reward*predicate
            sum_predicate += predicate
            
        if sum_predicate == 0:
            return 0
            
        return sum_rewards/sum_predicate
    
def get_reward(action):
    return np.random.normal(q_rewards[action],1)

def argmax(array):
    top_index = [0]
    top = array[0]
    
    for i in range(1,len(array)):
        if array[i] > top:
            top_index = [i]
            top = array[i]
        elif array[i] == top:
            top_index.append(i)
        
    np.random.shuffle(top_index)
        
    return top_index[0]

def epsilon_greedy_action_selection(t, actions, epsilon = 0):

    rand = np.random.rand(1)[0]

    if t <= 0 or epsilon >= rand:            
        return np.random.choice(10,1)[0]

    else:
        Q_t = []
        for i in ACTION:
            Q_t.append(sample_average(t,i, actions, epsilon))
        return argmax(Q_t)

In [ ]:
RUN = 100
STEP = 1000
ACTION = [i for i in range(10)]

q_rewards = []
for _ in range(10):
    tmp = np.random.normal(0,1)
    q_rewards.append(tmp)
    
optimal_action = np.argmax(q_rewards)

average_greedy_reward = [0]*STEP
e1_average_greedy_reward = [0]*STEP
e2_average_greedy_reward = [0]*STEP

greedy_optimal_action_ratio = [0]*STEP
e1_greedy_optimal_action_ratio = [0]*STEP
e2_greedy_optimal_action_ratio = [0]*STEP

## testbed
for t in tqdm_notebook(range(RUN)):
    
    ## greedy
    greedy_actions = []

    for i in range(STEP):
        greedy_actions.append(epsilon_greedy_action_selection(i, greedy_actions))
        if greedy_actions[i] == optimal_action:
            greedy_optimal_action_ratio[i] += 1/RUN
        average_greedy_reward[i] += get_reward(greedy_actions[i])/RUN
        
    ## ε = 0.1
    e1_greedy_actions = []

    epsilon1 = 0.1

    for i in range(STEP):
        e1_greedy_actions.append(epsilon_greedy_action_selection(i, e1_greedy_actions, epsilon1))
        if greedy_actions[i] == optimal_action:
            e1_greedy_optimal_action_ratio[i] += 1/RUN
        e1_average_greedy_reward[i] += get_reward(e1_greedy_actions[i])/RUN

    ## ε = 0.01
    e2_greedy_actions = []

    epsilon2 = 0.01

    for i in range(STEP):
        e2_greedy_actions.append(epsilon_greedy_action_selection(i, e2_greedy_actions, epsilon2))
        if greedy_actions[i] == optimal_action:
            e2_greedy_optimal_action_ratio[i] += 1/RUN
        e2_average_greedy_reward[i] += get_reward(e2_greedy_actions[i])/RUN

In [ ]:
plt.plot(np.arange(STEP), average_greedy_reward, 'g-', np.arange(STEP), e1_average_greedy_reward, 'b-', np.arange(STEP), e2_average_greedy_reward, 'r-')
plt.xlabel('steps')
plt.ylabel('average reward')
plt.show()

In [ ]:
plt.plot(np.arange(STEP), greedy_optimal_action_ratio, 'g-', np.arange(STEP), e1_greedy_optimal_action_ratio, 'b-', np.arange(STEP), e2_greedy_optimal_action_ratio, 'r-')
plt.xlabel('steps')
plt.ylabel('% optimal action')
plt.show()

Waste memory, High complexity